In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part 

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()


In [ ]:
# check connection to table
reviews_df = session.table('TEST.DATAMART.GOOGLE_PLACE_REVIEW') 
reviews_df.show(5)

Untuk Enable Cortex dan mengizinkan wilayah Snowflake mana pun yang mendukung permintaan inferensi lintas wilayah untuk memproses. tetapkan parameter ke 'ANY_REGION'.

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';

In [ ]:
from snowflake.snowpark.functions import col

filtered_reviews_df = reviews_df.filter(F.col('REVIEW_TRANSLATED_TEXT').is_not_null())

# Step 2: Define the prompt template for summary generation
prompt_template = """
Summarize the following Google Places user reviews for a logistics company into short, concise statements that highlight the key experiences using only the following categories:
"Fast Delivery," "Late Delivery," "Damaged Package," "Lost Package," "Unresponsive Customer Service," "Inaccurate Tracking Information," "High Shipping Costs," "Poor Packaging," "Friendly Staff," and "Failed Delivery Attempts."

Make sure each summary reflects the most important aspect of the user experience related to the logistics and shipping service.

Include the user rating and how it correlates to their experience in the summary.

Answers should only be strings in the format json: [{"Summary": "[insert summary of user experience]." "Sentiment": "[positive, negative, or neutral]." "Key Experience": "[relevant categories, separated by commas]"}]
"""

# Step 3: Create the CATEGORY_SENTIMENT column using Cortex and prompt template
reviews_with_sentiment_df = filtered_reviews_df.withColumn(
    'CATEGORY_SENTIMENT',
    cortex.Complete(
        'llama3.1-70b',
        F.expr(f"concat('{prompt_template}', 'Review: ', REVIEW_TRANSLATED_TEXT, '\\nRating: ', RATING, '\\nAnswer:[/INST]')")
    )
)

# Step 4: Display the resulting DataFrame
reviews_with_sentiment_df.show(10)
# Step 5 : Write data to table
reviews_with_sentiment_df.write.mode("overwrite").save_as_table("TEST.DATAMART.REVIEW_GOOGLE")


In [ ]:

# using mistral-large-2
from snowflake.snowpark.functions import col

filtered_reviews_df = reviews_df.filter(F.col('REVIEW_TRANSLATED_TEXT').is_not_null())

# Step 2: Define the prompt template for summary generation
prompt_template = """
Summarize the following Google Places user reviews for a logistics company into short, concise statements that highlight the key experiences using only the following categories:
"Fast Delivery," "Late Delivery," "Damaged Package," "Lost Package," "Unresponsive Customer Service," "Inaccurate Tracking Information," "High Shipping Costs," "Poor Packaging," "Friendly Staff," and "Failed Delivery Attempts."

Make sure each summary reflects the most important aspect of the user experience related to the logistics and shipping service.

Include the user rating and how it correlates to their experience in the summary.

Answers should only be strings in the format json: [{"Summary": "[insert summary of user experience]." "Sentiment": "[positive, negative, or neutral]." "Key Experience": "[relevant categories, separated by commas]"}]
"""

# Step 3: Create the CATEGORY_SENTIMENT column using Cortex and prompt template
reviews_with_sentiment_df = filtered_reviews_df.withColumn(
    'CATEGORY_SENTIMENT_2',
    cortex.Complete(
        'mistral-large2',
        F.expr(f"concat('{prompt_template}', 'Review: ', REVIEW_TRANSLATED_TEXT, '\\nRating: ', RATING)")
    )
)

# Step 4: Display the resulting DataFrame
reviews_with_sentiment_df.show(10)
selected_columns_df = reviews_with_sentiment_df.select('REVIEW_ID', 'CATEGORY_SENTIMENT_2')
pandasdf = selected_columns_df

pandasdf.show(5)
customer_wrt = pandasdf.write.mode("overwrite").save_as_table("TEST.DATAMART.GOOGLE_SUMMARY_2" ,table_type="transient")